#Load full e-commerce dataset

In [0]:
events_nov = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",
    header=True,
    inferSchema=True
)

events_oct = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",
    header=True,
    inferSchema=True
)

#Combine Data

In [0]:
events = events_oct.unionByName(events_nov)


#Validate Load

In [0]:
events.printSchema()
events.show(5)
events.count()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|    

109950743

#Create Derived Tables (Aggregations)

In [0]:
from pyspark.sql import functions as F

product_revenue = (
    events
    .filter(F.col("event_type") == "purchase")
    .groupBy("product_id", "category_code", "brand")
    .agg(F.sum("price").alias("total_revenue"))
)


#Left Join

In [0]:
events_left_join = (
    events.alias("e")
    .join(product_revenue.alias("pr"), on="product_id", how="left")
)

events_left_join.select(
    "product_id", "e.brand", "price", "total_revenue"
).show(10)

+----------+--------+------+------------------+
|product_id|   brand| price|     total_revenue|
+----------+--------+------+------------------+
|  28719074|   baden|102.71|              NULL|
|   1004545|  huawei|566.01|18967.559999999998|
|  31500053|luminarc| 41.16|           1000.69|
|   1480613|  pulser|908.62|           3634.48|
|   3900821|    aqua|  33.2|           2562.89|
|   1307067|  lenovo|251.74|302221.93999999994|
|  17200506|    NULL| 543.1| 5775.780000000001|
|  44600062|shiseido| 35.79|            107.37|
|   2900536|elenberg| 51.46| 48853.73000000002|
|  17300353|   creed|380.96| 37294.50000000002|
+----------+--------+------+------------------+
only showing top 10 rows


#Inner Join

In [0]:
events_inner_join = (
    events
    .join(product_revenue, on="product_id", how="inner")
)

events_inner_join.count()

105428916

#Right Join

In [0]:
events_right_join = (
    events
    .join(product_revenue, on="product_id", how="right")
)

events_right_join.select(
    "product_id", "total_revenue", "event_type"
).show(10)

+----------+------------------+----------+
|product_id|     total_revenue|event_type|
+----------+------------------+----------+
|   8500290|14674.029999999997|      view|
|   8500290|14674.029999999997|      view|
|   8500290|14674.029999999997|  purchase|
|   8500290|14674.029999999997|      view|
|   8500290|14674.029999999997|      view|
|   8500290|14674.029999999997|      view|
|   8500290|14674.029999999997|      view|
|   8500290|14674.029999999997|      view|
|   8500290|14674.029999999997|      view|
|   8500290|14674.029999999997|      view|
+----------+------------------+----------+
only showing top 10 rows


#Outer Join

In [0]:
events_outer_join = (
    events
    .join(product_revenue, on="product_id", how="outer")
)

events_outer_join.select(
    "product_id", "price", "total_revenue"
).show(10)

+----------+------+-----------------+
|product_id| price|    total_revenue|
+----------+------+-----------------+
|   1005159|231.41|763047.4699999999|
|   1005159|231.41|763047.4699999999|
|   1005159|231.41|763047.4699999999|
|   5701087| 51.48|5245.850000000002|
|   5701087| 51.48|5245.850000000002|
|   5701087| 51.48|5245.850000000002|
|   5701087| 51.48|5245.850000000002|
|  26402159|297.36|           872.98|
|   1005159|231.41|763047.4699999999|
|  26402159|297.36|           872.98|
+----------+------+-----------------+
only showing top 10 rows


#Window Functions

In [0]:
# import required modules
from pyspark.sql import functions as F
from pyspark.sql.window import Window

#Define a Window Specification
user_window = (
    Window
    .partitionBy("user_id")
    .orderBy("event_time")
)

# Running Total of Events per User
events_with_running_events = (
    events
    .withColumn(
        "cumulative_events",
        F.count("*").over(user_window)
    )
)

events_with_running_events.select(
    "user_id", "event_time", "cumulative_events"
).show(10)

+--------+-------------------+-----------------+
| user_id|         event_time|cumulative_events|
+--------+-------------------+-----------------+
|65800726|2019-11-27 04:33:16|                1|
|65800726|2019-11-27 04:35:24|                2|
|81255481|2019-11-08 07:44:45|                1|
|81255481|2019-11-21 14:11:26|                2|
|82079354|2019-11-28 04:58:01|                1|
|82079354|2019-11-28 04:58:22|                2|
|82079354|2019-11-28 04:59:29|                3|
|82079354|2019-11-28 04:59:54|                4|
|82079354|2019-11-28 05:00:22|                5|
|82079354|2019-11-28 05:00:47|                6|
+--------+-------------------+-----------------+
only showing top 10 rows


In [0]:
# Ranking Events per User (By Time)
events_with_rank = (
    events
    .withColumn(
        "event_rank",
        F.row_number().over(user_window)
    )
)

events_with_rank.select(
    "user_id", "event_time", "event_rank"
).show(10)  

+--------+-------------------+----------+
| user_id|         event_time|event_rank|
+--------+-------------------+----------+
|65800726|2019-11-27 04:33:16|         1|
|65800726|2019-11-27 04:35:24|         2|
|81255481|2019-11-08 07:44:45|         1|
|81255481|2019-11-21 14:11:26|         2|
|82079354|2019-11-28 04:58:01|         1|
|82079354|2019-11-28 04:58:22|         2|
|82079354|2019-11-28 04:59:29|         3|
|82079354|2019-11-28 04:59:54|         4|
|82079354|2019-11-28 05:00:22|         5|
|82079354|2019-11-28 05:00:47|         6|
+--------+-------------------+----------+
only showing top 10 rows


#Create Derived Features

In [0]:
# Convert event_type into a numeric signal.
events_features = (
    events
    .withColumn(
        "is_purchase",
        F.when(F.col("event_type") == "purchase", 1).otherwise(0)
    )
)
# Price Bucket (Low / Medium / High)
events_features = (
    events_features
    .withColumn(
        "price_bucket",
        F.when(F.col("price") < 50, "low")
         .when(F.col("price") < 200, "medium")
         .otherwise("high")
    )
)
# Count how many events happened in a session.
session_window = Window.partitionBy("user_session")

events_features = (
    events_features
    .withColumn(
        "session_event_count",
        F.count("*").over(session_window)
    )
)
# User-Level Purchase Ratio
user_metrics = (
    events_features
    .groupBy("user_id")
    .agg(
        F.count("*").alias("total_events"),
        F.sum("is_purchase").alias("total_purchases")
    )
)
# Compute purchase ratio
user_metrics = (
    user_metrics
    .withColumn(
        "purchase_ratio",
        F.col("total_purchases") / F.col("total_events")
    )
)
# Join User Features Back to Events
events_final = (
    events_features
    .join(user_metrics, on="user_id", how="left")
)
# Validate Derived Features
events_final.select(
    "user_id",
    "event_type",
    "is_purchase",
    "price",
    "price_bucket",
    "session_event_count",
    "purchase_ratio"
).show(10)

+---------+----------+-----------+------+------------+-------------------+--------------------+
|  user_id|event_type|is_purchase| price|price_bucket|session_event_count|      purchase_ratio|
+---------+----------+-----------+------+------------+-------------------+--------------------+
|515618310|      view|          0|360.34|        high|                  5|0.005847953216374269|
|556677167|      view|          0|203.07|        high|                  2|           0.0390625|
|541806858|      view|          0|157.02|      medium|                  3|                 0.0|
|531956094|      view|          0| 38.35|         low|                  4| 0.02127659574468085|
|557948248|      view|          0|308.63|        high|                  6|                 0.0|
|540247397|      view|          0|179.26|      medium|                  2|                 0.0|
|531142295|      view|          0|427.78|        high|                  6|                 0.0|
|559522167|      view|          0|229.71